## Look at methylation levels using the Karaka v1 primary assembly:
- Modified basecalling done by Ignacio Carvajal
- Summarise global methylation levels
- Generate percent methylaiton per 100k windows for Chen Wu's circos plot
- Some of these scripts were adapted from code provided in this Repo: https://github.com/yusmiatiliau/Plant_skimseq_methylation

# 00.merge bams

In [ ]:
WKDIR=${BASE}/methylation
OUT=${BASE}/methylation/00.merge_bams

mkdir -p -v ${OUT}
cd ${WKDIR}

ml samtools/1.20

sbatch << EOF
#!/bin/bash
#SBATCH -J merge
#SBATCH --cpus-per-task=6
#SBATCH --mem-per-cpu=104G
#SBATCH --time=02:00:00
#SBATCH -o $LOG/hrasrb_%J.out
#SBATCH -e $LOG/hrasrb_%J.err
#SBATCH --mail-user=Sarah.Bailey@plantandfood.co.nz
#SBATCH --mail-type=ALL

samtools merge -@ 6 -o ${OUT}/KarakaPN_ONT_merged.bam \
    ${IN}/KarakaPN_ONT.bam \
    ${IN}/KarakaPN_ONT2.bam
EOF

module unload samtools

# 01.align

In [3]:
WKDIR=${BASE}/methylation
OUT=${BASE}/methylation/01.align
REF=${REFERENCE_PATH}/karaka_v1.primary.chromosome-only.fa
mkdir -p -v ${OUT}
cd ${OUT}

sbatch --dependency=afterok:9327201 << EOF
#!/bin/bash
#SBATCH -J dorado_align
#SBATCH --cpus-per-task=6
#SBATCH --mem-per-cpu=48G
#SBATCH --time=12:00:00
#SBATCH -o $LOG/hrasrb_%J.out
#SBATCH -e $LOG/hrasrb_%J.err
#SBATCH --mail-user=Sarah.Bailey@plantandfood.co.nz
#SBATCH --mail-type=ALL

${MODULE_PATH}/dorado-0.9.6-linux-x64/bin/dorado aligner ${REF} \
    ${WKDIR}/00.merge_bams/KarakaPN_ONT_merged.bam --threads 6 > ${OUT}/karaka_v1.primary.5mC.bam

EOF

mkdir: created directory '/powerplant/workspace/hrasrb/Karaka/methylation/01.align'
Submitted batch job 9327202


# 02.sort and index

In [1]:
WKDIR=${BASE}/methylation
OUT=${BASE}/methylation/01.align

mkdir -p -v ${OUT}
cd ${OUT}

ml samtools/1.20
sbatch << EOF
#!/bin/bash
#SBATCH -J filt
#SBATCH --cpus-per-task=4
#SBATCH --mem-per-cpu=64G
#SBATCH --time=01:00:00
#SBATCH -o $LOG/hrasrb_%J.out
#SBATCH -e $LOG/hrasrb_%J.err
#SBATCH --mail-user=Sarah.Bailey@plantandfood.co.nz
#SBATCH --mail-type=ALL

# samtools sort -@ 2 ./karaka_v1.primary.5mC.bam -O BAM > ./karaka_v1.primary.5mC.sort.bam
# samtools index -@ 2 ./karaka_v1.primary.5mC.sort.bam
samtools view -F 260 -@4 -hb ./karaka_v1.primary.5mC.sort.bam > ./karaka_v1.primary.5mC.filt.bam
samtools index -@4 ./karaka_v1.primary.5mC.filt.bam
rm ./karaka_v1.primary.5mC.sort.bam
EOF

module unload samtools

Submitted batch job 9329686


# 03.check call accuracy

In [8]:
WKDIR=${BASE}/methylation
OUT=${BASE}/methylation/02.check_call_accuracy

mkdir -p ${OUT}
cd ${OUT}

sbatch << EOF
#!/bin/bash
#SBATCH -J sample_probs
#SBATCH --cpus-per-task=2
#SBATCH --mem-per-cpu=16G
#SBATCH --time=12:00:00
#SBATCH -o $LOG/hrasrb_%J.out
#SBATCH -e $LOG/hrasrb_%J.err
#SBATCH --mail-user=Sarah.Bailey@plantandfood.co.nz
#SBATCH --mail-type=ALL

${MODULE_PATH}/dist_modkit_v0.5.0_5120ef7/modkit sample-probs ${WKDIR}/01.align/karaka_v1.primary.5mC.sort.bam --hist --out-dir ${OUT}

EOF

Submitted batch job 9327326


# 04.qualimap

In [2]:
ml apptainer/1.1

WKDIR=${BASE}/methylation
OUT=${BASE}/methylation/04.qualimap

mkdir -p ${OUT}
cd ${OUT}

sbatch --dependency=afterok:9329686 << EOF
#!/bin/bash -e

#SBATCH -J qualimap
#SBATCH -o ${LOG}/hrasrb_%J.out
#SBATCH -e ${LOG}/hrasrb_%J.err
#SBATCH --cpus-per-task=6
#SBATCH --mem=80G
#SBATCH --time=01:20:00
#SBATCH --mail-user=Sarah.Bailey@plantandfood.co.nz
#SBATCH --mail-type=ALL

export TMPDIR="/workspace/\$USER/tmp"
export APPTAINER_BINDPATH="\$APPTAINER_BINDPATH,\$TMPDIR:\$TMPDIR,\$TMPDIR:/tmp"
export SINGULARITY_CACHEDIR="/workspace/\$USER/my_modules/singularity"

${MODULE_PATH}/singularity/qualimap_2.3--hdfd78af_0.sif qualimap bamqc \
    -bam ${WKDIR}/01.align/karaka_v1.primary.5mC.filt.bam \
    -nt 6 \
    -outdir ${OUT} \
    -outfile karaka_v1.primary.5mC.filt.html \
    -outformat HTML \
    --java-mem-size=80G \
    --paint-chromosome-limits

EOF
module unload apptainer

Submitted batch job 9329688


# 05.modkit

In [4]:
WKDIR=${BASE}/methylation
OUT=${BASE}/methylation/05.modkit
REF=/output/genomic/plant/Corynocarpus/laevigatus/genome/v1/assembly/karaka_v1.primary.chromosome-only.fa

mkdir -p ${OUT}
cd ${OUT}

sbatch << EOF
#!/bin/bash -e

#SBATCH -J qualimap
#SBATCH -o ${LOG}/hrasrb_%J.out
#SBATCH -e ${LOG}/hrasrb_%J.err
#SBATCH --cpus-per-task=4
#SBATCH --mem=56G
#SBATCH --time=01:00:00
#SBATCH --mail-user=Sarah.Bailey@plantandfood.co.nz
#SBATCH --mail-type=ALL

${MODULE_PATH}/dist_modkit_v0.5.0_5120ef7/modkit pileup ${WKDIR}/01.align/karaka_v1.primary.5mC.filt.bam "./karaka_v1.primary.5mC_t90_allChrs.pileup.bed" --ref "${REF}" --motif CG 0 --motif CHG 0 --motif CHH 0 -t 4 --log-filepath ./t90.log --filter-threshold C:0.85 --mod-threshold m:0.90
sed -i 's| |\t|g' "./karaka_v1.primary.5mC_t90_allChrs.pileup.bed"

EOF


Submitted batch job 9329858


## Run split_bed
first split by methylation type, then by chr and then compress and index

In [7]:
WKDIR=${BASE}/methylation
OUT=${BASE}/methylation/05.modkit
PREFIX=karaka_v1.primary.5mC_t90

mkdir -p ${OUT}
cd ${OUT}

ml htslib
sbatch << EOF
#!/bin/bash -e

#SBATCH -J compress_index
#SBATCH -o ${LOG}/hrasrb_%J.out
#SBATCH -e ${LOG}/hrasrb_%J.err
#SBATCH --cpus-per-task=1
#SBATCH --mem=32G
#SBATCH --time=01:00:00
#SBATCH --mail-user=Sarah.Bailey@plantandfood.co.nz
#SBATCH --mail-type=ALL

# echo "split unzipped bed files by context."
# awk -v P="${PREFIX}" '{fname=\$4; gsub(/,/, "_", fname); print > (fname "_" P "_allChrs.bed")}' "${PREFIX}_allChrs.pileup.bed"
# echo "${PREFIX} done."

# cat m_CG_0_${PREFIX}_allChrs.bed | \
#     awk -v S="${PREFIX}" -v M="CG" '{print > (\$1 "_" S "_" M ".bed")}'
# echo "Split CG context done"
# cat m_CHG_0_${PREFIX}_allChrs.bed | \
#     awk -v S="${PREFIX}" -v M="CHG" '{print > (\$1 "_" S "_" M ".bed")}'
# echo "Split CHG context done"
# cat m_CHH_0_${PREFIX}_allChrs.bed | \
#     awk -v S="${PREFIX}" -v M="CHH" '{print > (\$1 "_" S "_" M ".bed")}'
# echo "Split CHH context done"

for c in chr{1..23}; do
    file="\${c}_${PREFIX}_CG.bed"
    bgzip -f "\$file"
    tabix -f -p bed "\${file}.gz"
done
echo "Compress and index CG context done."
for c in chr{1..23}; do
    file="\${c}_${PREFIX}_CHG.bed"
    bgzip -f "\$file"
    tabix -f -p bed "\${file}.gz"
done
echo "Compress and index CHG context done."
for c in chr{1..23}; do
    file="\${c}_${PREFIX}_CHH.bed"
    bgzip -f "\$file"
    tabix -f -p bed "\${file}.gz"
done
echo "Compress and index CHH context done."

EOF

Submitted batch job 9331258


## Run filterMethylation_{min_cov}x.py

In [4]:
WKDIR=${BASE}/methylation/06.filter

cd ${WKDIR}

ml pfr-python3

sbatch << EOF
#!/bin/bash -e

#SBATCH -J plotMethylation
#SBATCH -o ${LOG}/hrasrb_%J.out
#SBATCH -e ${LOG}/hrasrb_%J.err
#SBATCH --cpus-per-task=2
#SBATCH --mem=16G
#SBATCH --time=02:00:00
#SBATCH --mail-user=Sarah.Bailey@plantandfood.co.nz
#SBATCH --mail-type=ALL

python3 ${REPO_PATH}/High-quality-genomes/Karaka/filterMethylation_5x.py

EOF
module unload pfr-python3

Submitted batch job 9336313


In [9]:
# combine chr files

WKDIR=${BASE}/methylation/06.filter

cd ${WKDIR}
# cut -f 1 ${REFERENCE_PATH}/karaka_v1.primary.chromosome-only.fa.fai > ids.txt

module load htslib/1.20

# create and submit bash script
sbatch << EOF
#!/bin/bash -e

#SBATCH -J create_allChrs # change
#SBATCH --output=${LOG}/hrasrb_%j.out
#SBATCH --error=${LOG}/hrasrb_%j.err
#SBATCH --mail-user=Sarah.Bailey@plantandfood.co.nz
#SBATCH --mail-type=ALL
#SBATCH --time=01:30:00 # Walltime
#SBATCH --mem=32G

while read -r chr; do
    prefix="\${chr}_karaka_v1.primary.5mC_t90_CHH_5x"
    bgzip -d -f "${WKDIR}/\${prefix}.bed.gz"
    cat "${WKDIR}/\${prefix}.bed" >> karaka_v1.primary.5mC_t90_CHH.bed
done < ids.txt
echo "combined chrs."

sort -k1,1V -k2,2n karaka_v1.primary.5mC_t90_CHH.bed
echo "sorted."

bgzip -f karaka_v1.primary.5mC_t90_CHH.bed
echo "compressed."

tabix -f -p bed karaka_v1.primary.5mC_t90_CHH.bed.gz
echo "indexed."

EOF
module unload htslib

Submitted batch job 9343149


In [10]:
WKDIR=${BASE}/methylation/07.modkit_stats
TAG=karaka_v1
mkdir -p -v ${WKDIR}
cd ${WKDIR}

## If file already exists delete it, otherwise text gets appended to existing file
FILE=./bed.fofn
if [ -f "$FILE" ] ; then
    rm "$FILE"
fi

cat <<'EOF' >> ./bed.fofn
${BASE}/methylation/07.modkit_stats/karaka_v1.primary.5mC_t90_CG.bed.gz
${BASE}/methylation/07.modkit_stats/karaka_v1.primary.5mC_t90_CHG.bed.gz
${BASE}/methylation/07.modkit_stats/karaka_v1.primary.5mC_t90_CHH.bed.gz
EOF

ml htslib
cat $FILE | while read line
do
    PREFIX=$(basename ${line} .bed.gz)
    echo "${MODULE_PATH}/dist_modkit_v0.5.0_5120ef7/modkit stats --log-filepath "${PREFIX}_stats.log" --no-header --mod-codes "m" --regions ${IN}/32.circosPlot_paper/windows_chr.100k.txt -o "${PREFIX}.stats.100k.bed" ${line}"
done | abatch -j probs_array --time 00:20:00 --mem 2G --cpus-per-task=2 --mail-user=Sarah.Bailey@plantandfood.co.nz --mail-type=ALL | sbatch --dependency=afterok:9343149
module unload htslib

mkdir: created directory '/powerplant/workspace/hrasrb/Karaka/methylation/07.modkit_stats'
SBATCH_ARGS: --time 00:20:00 --mem 2G --cpus-per-task=2 --mail-user=Sarah.Bailey@plantandfood.co.nz --mail-type=ALL
JOB_ARRAY_NAME: probs_array
GROUP_SIZE: 1
NUM_COMMANDS: 3
Submitted batch job 9343163
